In [1]:
from model import ShallowCNN, ShallowVGG
import torchvision

# vgg16
vgg16 = torchvision.models.vgg16()
# shallow CNN
shallowCNN = ShallowCNN()
# shallow VGG
shallowVGG = ShallowVGG()

/Users/wonseo/miniforge3/envs/ml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Count the parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("VGG16's trainable parameter: ", count_parameters(vgg16))
print("Shallow CNN's trainable parameter: ", count_parameters(shallowCNN))
print("Shallow VGG's parameter", count_parameters(shallowVGG))

VGG16's trainable parameter:  138357544
Shallow CNN's trainable parameter:  260160
Shallow VGG's parameter 265191


In [32]:
# Load pretrained VGG16's weight
vgg_dict = vgg16.state_dict()
print("-----VGG16's layers-----")
print(vgg_dict.keys())

shallow_dict = shallowCNN.state_dict()
print("-----Shallow CNN's layers-----")
print(shallow_dict.keys())

pretrained_dict = {k: v for k, v in vgg_dict.items() if k in shallow_dict}
print("-----Filtered layers-----")
print("update only this layers")
print(pretrained_dict.keys())

shallow_dict.update(pretrained_dict) 

-----VGG16's layers-----
odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight', 'classifier.6.bias'])
-----Shallow CNN's layers-----
odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias'])
-----Filtered layers-----
update only this layers
dict_keys(['features.0.weight', 'featu

In [49]:
# Test model working
import cv2
import numpy as np
import torch

sampleImage = cv2.imread("./Plant_leave_diseases_dataset_with_augmentation/Apple___Apple_scab/image (1).JPG")
sampleImage = np.transpose(sampleImage, (2, 0, 1))
sampleImage = torch.Tensor(sampleImage)

output = shallowCNN(sampleImage)
print(output.shape)
print(output)

torch.Size([128])
tensor([7.2555e+00, 4.8985e-02, 3.8492e-01, 1.4317e+00, 3.4436e+00, 0.0000e+00,
        1.4937e+00, 2.7819e-02, 3.7448e+00, 9.6746e-01, 7.1532e+00, 2.5898e+00,
        3.2852e-03, 2.5271e-02, 1.3415e+00, 6.0928e-01, 1.8503e-01, 4.9054e-02,
        2.0477e-03, 4.4897e+00, 3.2683e-03, 1.6110e-03, 1.1250e-01, 9.8819e-02,
        0.0000e+00, 4.0479e-01, 9.4730e+00, 4.2584e-02, 5.1130e+00, 4.4799e-02,
        8.0742e-03, 1.6708e-02, 1.6456e-03, 6.9410e+00, 1.8681e-02, 4.4142e+00,
        2.8696e-03, 0.0000e+00, 4.1467e-02, 1.5366e-04, 1.4220e-02, 2.3933e-02,
        2.8302e+00, 2.0717e+00, 2.4845e-02, 9.9305e+00, 1.9894e-02, 9.2752e-03,
        9.7521e-01, 2.7569e+00, 1.9858e+00, 1.8247e-02, 4.8091e+00, 2.7693e-02,
        6.2683e+00, 4.5448e-02, 2.3253e-03, 2.4652e-04, 3.9430e-01, 0.0000e+00,
        1.9213e-01, 0.0000e+00, 5.1409e+00, 8.6206e-06, 4.7371e+00, 1.9745e+00,
        5.1371e+00, 9.2254e-04, 5.1419e-03, 6.1720e+00, 1.3487e-02, 2.6406e+00,
        8.2571e+00, 3.

In [50]:
# PCA: 99% reserved variance
from sklearn.decomposition import PCA
from torch import detach

output = output.detach()
output = output.numpy()
pca = PCA(n_components=0.99)
output_pca = pca.fit_transform(output.reshape(1, -1))

[0.]


/Users/wonseo/miniforge3/envs/ml/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:525: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


array([0.])

In [10]:
# Random Foreset
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score

x_train = ...
y_train = ...
x_test = ...
y_test = ...

randomForest = RandomForestClassifier(random_state=123)
randomForest.fit(x_train,y_train)
y_pred_rf = randomForest.predict(x_test)

print("-----Random Forest result-----")
print("accuracy is ", accuracy_score(y_test, y_pred_rf))
print("recall is ", recall_score(y_test, y_pred_rf))
print("precision is ", precision_score(y_test, y_pred_rf))
print("f1 is ", f1_score(y_test, y_pred_rf))

# SVM: Gaussian kernel
from sklearn import svm

svm = svm.SVC(kernel="rbf")
svm.fit(x_train, y_train)
y_pred_svm = svm.predict(x_test)

print("-----SVM result-----")
print("accuracy is ", accuracy_score(y_test, y_pred_svm))
print("recall is ", recall_score(y_test, y_pred_svm))
print("precision is ", precision_score(y_test, y_pred_svm))
print("f1 is ", f1_score(y_test, y_pred_svm))

TypeError: Singleton array array(1) cannot be considered a valid collection.

In [1]:
from model import ShallowVGG

shallowVGG = ShallowVGG()

/Users/wonseo/miniforge3/envs/ml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Test model working
import cv2
import numpy as np
import torch

sampleImage = cv2.imread("./Plant_leave_diseases_dataset_with_augmentation/Apple___Apple_scab/image (1).JPG")
sampleImage = np.transpose(sampleImage, (2, 0, 1))
sampleImage = torch.Tensor(sampleImage)

output = shallowVGG(sampleImage)
print(output.shape)
print(output)
print(torch.argmax(output).item()+1)

torch.Size([1, 39])
tensor([[1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07,
         1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07,
         1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07,
         1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07,
         1.7687e-07, 1.7687e-07, 1.7687e-07, 1.6659e-01, 1.7687e-07, 1.7687e-07,
         1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 1.7687e-07, 8.3341e-01,
         1.7687e-07, 1.7687e-07, 1.7687e-07]], grad_fn=<SoftmaxBackward0>)
36
